In [2]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc, html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64
JupyterDash.infer_jupyter_proxy_config()

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from CRUD_Python_Module import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name

username = "aacuser"
password = "ChooseAStrongPassword123!"

# Connect to database via CRUD Module
db = AnimalShelter(username, password)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
def fetch_df(query: dict | None = None) -> pd.DataFrame:
    """Run a read query and return a clean DataFrame without _id."""
    data = list(db.read(query or {}))
    df = pd.DataFrame(data)


# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
    if "_id" in df.columns:
        df.drop(columns=["_id"], inplace=True)
    return df

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)
df_all = fetch_df({})

def build_query(filter_type: str) -> dict:
    """
    Build a Mongo query for the selected rescue type.
    Uses case insensitive regex so 'Mix' variants match too.
    """
    if filter_type == "water":
        breeds = ["Labrador Retriever", "Chesapeake Bay Retriever", "Newfoundland"]
        sex    = "Intact Female"
        age    = {"$gte": 26, "$lte": 156}

    elif filter_type == "mountain":
        breeds = ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog",
                  "Siberian Husky", "Rottweiler"]
        sex    = "Intact Male"
        age    = {"$gte": 26, "$lte": 156}

    elif filter_type == "disaster":
        breeds = ["Doberman Pinscher", "German Shepherd", "Golden Retriever",
                  "Bloodhound", "Rottweiler"]
        sex    = "Intact Male"
        age    = {"$gte": 20, "$lte": 300}

    else:
        return {}  # reset (all)

    pattern = "|".join([f"({b})" for b in breeds])
    return {
        "animal_type": "Dog",
        "breed": {"$regex": pattern, "$options": "i"},
        "sex_upon_outcome": sex,
        "age_upon_outcome_in_weeks": age,
    }

#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

#FIX ME Add in Grazioso Salvare’s logo
LOGO_PATH = "Grazioso Salvare Logo.png"
with open(LOGO_PATH, "rb") as f:
    encoded_logo = base64.b64encode(f.read()).decode()

brand_bar = html.Div(
    [
        html.Img(
            src=f"data:image/png;base64,{encoded_logo}",
            style={"height": "64px", "marginRight": "12px"},
            alt="Grazioso Salvare",
        ),
        html.H2("Grazioso Salvare Dashboard — Jordan Charlot", style={"margin": "0"}),
    ],
    style={"display": "flex", "alignItems": "center", "gap": "12px"},
)

app.layout = html.Div(
    [
        brand_bar,
        html.Hr(),

        # Interactive filter options
        html.Div(
            [
                html.Label("Rescue Type"),
                dcc.RadioItems(
                    id="filter-type",
                    options=[
                        {"label": "Reset (All Dogs)",           "value": "reset"},
                        {"label": "Water Rescue",               "value": "water"},
                        {"label": "Mountain / Wilderness",      "value": "mountain"},
                        {"label": "Disaster / Individual Track","value": "disaster"},
                    ],
                    value="reset",
                    labelStyle={"display": "inline-block", "marginRight": "18px"},
                    inputStyle={"marginRight": "6px"},
                ),
            ],
            style={"marginBottom": "8px"},
        ),

        # Interactive DataTable
        dash_table.DataTable(
            id="datatable-id",
            columns=[{"name": c, "id": c, "deletable": False, "selectable": True} for c in df_all.columns],
            data=df_all.to_dict("records"),
            row_selectable="single",
            selected_rows=[0],
            page_size=10,
            sort_action="native",
            filter_action="native",
            style_table={"overflowX": "auto", "maxHeight": "420px", "overflowY": "auto"},
            style_cell={"textAlign": "left", "minWidth": "120px", "whiteSpace": "normal"},
            style_header={"fontWeight": "bold"},
        ),

        html.Br(),
        html.Hr(),

        # Charts row: pie + map side-by-side
        html.Div(
            [
                html.Div(id="graph-id", style={"flex": 1, "minWidth": "420px", "marginRight": "16px"}),
                html.Div(id="map-id",   style={"flex": 1, "minWidth": "520px"}),
            ],
            style={"display": "flex", "flexWrap": "wrap"},
        ),
    ],
    style={"padding": "12px"},
)

#############################################
# Interaction Between Components / Controller
#############################################



# Update DataTable when filter changes   
@app.callback(Output('datatable-id','data'),
              [Input('filter-type', 'value')])

def update_dashboard(filter_type):
    q = build_query(filter_type)
    df = fetch_df(q)
    return df.to_dict("records") if not df.empty else []

# Display the breeds of animal based on quantity represented in
# the data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])

def update_graphs(viewData):
    dff = pd.DataFrame.from_dict(viewData or [])
    if dff.empty or "outcome_type" not in dff.columns:
        fig = px.scatter(title="No data")
    else:
        fig = px.pie(dff, names="outcome_type", title="Outcome Type distribution", hole=0.35)
        fig.update_traces(textposition="inside", textinfo="percent+label")
    return [dcc.Graph(figure=fig)]

    
#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    selected_columns = selected_columns or []   # handle None on first render
    return [{
        "if": {"column_id": col},
        "background_color": "#D2F3FF"
    } for col in selected_columns]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):  
    if viewData is None:
        return
    elif index is None:
        return
    
    dff = pd.DataFrame.from_dict(viewData)
    # Because we only allow single row selection, the list can be converted to a row index here
    if index is None:
        row = 0
    else: 
        row = index[0]
        
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            # Column 13 and 14 define the grid-coordinates for the map
            # Column 4 defines the breed for the animal
            # Column 9 defines the name of the animal
            dl.Marker(position=[dff.iloc[row,13],dff.iloc[row,14]], children=[
                dl.Tooltip(dff.iloc[row,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row,9])
                ])
            ])
        ])
    ]


# Run app and display result in jupyterlab mode, note, if you have previously run a prior app, the default port of 8050 may not be available, if so, try setting an alternate port.
# Previous Dash app is still running on the default port 8050
app.run_server(mode="inline", port=8051)